This notebook contains a few examples for bcc Fe to illustrate how pyVASP works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp() or self.submit_job()

In [1]:
from subprocess import run
import numpy as np
from pyVASP.code.main import pyVASP

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

import nglview

In [2]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# set calculation

set crystal structure

In [3]:
a = 3.95
alpha = 90
structure = crystal(('Ga', 'Mn', 'N'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                       spacegroup=221,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [4]:
# create vasp class
vasp = pyVASP(verbose="low", executable_path=executable_path, potential_path=potential_path, seed_mag=23)

In [5]:
# set magdirs for a triangular AFM state
value_of_m = 0.99

Dtheta = 120 * np.pi/180
theta = -Dtheta

number_of_atoms = len(structure)
magdirs   = []
ms    = []
# B_CONSTRs = []
for i in range(number_of_atoms):
   if 'Mn' == structure.get_chemical_symbols()[i]:
      theta += Dtheta
      magdirs.append([np.cos(theta), np.sin(theta), 0])
      ms.append(value_of_m)
      # B_CONSTRs.append([0.0, 0.0, 0.0])
   else:
      magdirs.append([0.0, 0.0, 0.0])
      ms.append(1)
      # B_CONSTRs.append([0.0, 0.0, 0.0])

structure.new_array("magdirs", magdirs, dtype=float)
structure.new_array("ms", ms, dtype=float)
# structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

In [6]:
# first prepare bfields (constraints)
vasp.prepare_bfields(I_CONSTRAINED_M="4", LAMBDA="1")
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize structure

In [7]:
view = nglview.show_ase(structure)
view.add_unitcell()
view

NGLWidget()

visualize some inputs

In [8]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Ga,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)",1.00,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
3,Mn,"(1.975, 1.975, 0.0)","(-0.5000000000000004, -0.8660254037844384, 0.0)",0.99,100.0,"(-0.5550793468129792, -0.8313408576381612, 0.0...","(0.0, 0.0, 0.0)"
2,Mn,"(1.975, 0.0, 1.975)","(-0.4999999999999998, 0.8660254037844387, 0.0)",0.99,100.0,"(-0.6366653015755763, 0.7703770181056809, 0.03...","(0.0, 0.0, 0.0)"
1,Mn,"(0.0, 1.975, 1.975)","(1.0, 0.0, 0.0)",0.99,100.0,"(0.9927164444766768, -0.039408363142266575, -0...","(0.0, 0.0, 0.0)"
4,N,"(1.975, 1.975, 1.975)","(0.0, 0.0, 0.0)",1.00,inf,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"


In [9]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0      [0.0000000000, 0.0000000000, 0.0000000000]
3    [-0.5550793468, -0.8313408576, 0.0275553473]
2     [-0.6366653016, 0.7703770181, 0.0343007834]
1    [0.9927164445, -0.0394083631, -0.1138465712]
4      [0.0000000000, 0.0000000000, 0.0000000000]
Name: magmoms, dtype: object

In [10]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[3.95 0.   0.  ]
 [0.   3.95 0.  ]
 [0.   0.   3.95]]

species = ['Ga', 'Mn', 'N']

elements = 0    Ga
3    Mn
2    Mn
1    Mn
4     N
Name: elements, dtype: object
